#Download from URL

In [1]:
import requests

In [2]:
page = requests.get("https://bundestag.de/apps/na/na/fraktion.form?id=200")

In [3]:
html = page.content

In [4]:
page.status_code

200

#Parse HTML for votings

In [5]:
import bs4
import re

In [6]:
soup = bs4.BeautifulSoup(html)

In [7]:
vec_div_namentlichBlock = soup.find_all("div", class_="namentlichBlock")

In [8]:
vec_li = []
for div_namentlichBlock in vec_div_namentlichBlock:
    vec_li += div_namentlichBlock.find_all("li")

In [9]:
def type_voting(str):
    if(str == "Mit ja abgestimmt"):
        return 1
    elif(str == "Mit nein abgestimmt"):
        return -1
    elif(str == "Enthalten"):
        return 0
    elif(str == "Nicht anwesend"):
        return -10
    else:
        error("uuups ... what's that?")

In [10]:
votings_data = []
for li in vec_li:
    title = li.a["title"]
    vote_parts = re.search("([^:]+):([^:]+):([^():]+)",title)
    votings_data.append({
            "type": type_voting(vote_parts.group(1)),
            "name": vote_parts.group(2).strip(),
            "party": vote_parts.group(3).strip()
        })

In [11]:
sorted(votings_data, key=lambda k: k["name"])

[{'name': 'Ackermann, Jens', 'party': 'FDP', 'type': 1},
 {'name': 'Ahrendt, Christian', 'party': 'FDP', 'type': 1},
 {'name': 'Aigner, Ilse', 'party': 'CDU/CSU', 'type': 1},
 {'name': 'Aken, Jan van', 'party': 'Die Linke', 'type': -1},
 {'name': 'Alpers, Agnes', 'party': 'Die Linke', 'type': -1},
 {'name': 'Altmaier, Peter', 'party': 'CDU/CSU', 'type': -10},
 {'name': 'Andreae, Kerstin', 'party': 'B90/Grüne', 'type': 1},
 {'name': 'Arndt-Brauer, Ingrid', 'party': 'SPD', 'type': 1},
 {'name': 'Arnold, Rainer', 'party': 'SPD', 'type': 1},
 {'name': 'Aschenberg-Dugnus, Christine', 'party': 'FDP', 'type': 1},
 {'name': 'Aumer, Peter', 'party': 'CDU/CSU', 'type': 1},
 {'name': 'Bahr, Daniel', 'party': 'FDP', 'type': 1},
 {'name': 'Barchmann, Heinz-Joachim', 'party': 'SPD', 'type': 1},
 {'name': 'Bareiß, Thomas', 'party': 'CDU/CSU', 'type': 1},
 {'name': 'Barnett, Doris', 'party': 'SPD', 'type': 1},
 {'name': 'Bartels, Dr. Hans-Peter', 'party': 'SPD', 'type': 1},
 {'name': 'Barthel, Klaus',

#Parse HTML for voting meta data

In [12]:
meta = soup.find("div", class_="namentlichText")

In [13]:
meta

<div class="namentlichText">
<h2><span class="dachzeile">14.12.2012</span><br/>Bundeswehreinsatz in der Türkei</h2>
<p>Beschlussempfehlung des Auswärtigen Ausschusses zum Antrag der Bundesregierung über die Entsendung bewaffneter deutscher Streitkräfte zur Verstärkung der integrierten Luftverteidigung der NATO auf Ersuchen der Türkei (Drucksachen <a class="linkGeneric" href="http://dip.bundestag.de/btd/17/117/1711783.pdf" target="_blank" title="Die PDF-Datei Drucksache 17/11783 öffnet sich in einem neuen Fenster">17/11783</a>, <a class="linkGeneric" href="http://dip.bundestag.de/btd/17/118/1711892.pdf" target="_blank" title="Die PDF-Datei Drucksache 17/11892 öffnet sich in einem neuen Fenster">17/11892</a>)</p>
</div>

In [14]:
gs = re.search("([0-9.]{10})(.+)",meta.find("h2").text)

In [15]:
date = gs.group(1)
dps = re.search("([0-9]{2})\.([0-9]{2})\.([0-9]{4})", date)
date = dps.group(3)+"-"+dps.group(2)+"-"+dps.group(1)
title = gs.group(2)
sub = re.sub("\r\n", ". ", meta.p.text)

In [16]:
date,title,sub

('2012-12-14',
 'Bundeswehreinsatz in der Türkei',
 'Beschlussempfehlung des Auswärtigen Ausschusses zum Antrag der Bundesregierung über die Entsendung bewaffneter deutscher Streitkräfte zur Verstärkung der integrierten Luftverteidigung der NATO auf Ersuchen der Türkei (Drucksachen 17/11783, 17/11892)')

#Connect to Postgres

In [17]:
import psycopg2

In [18]:
db = psycopg2.connect("""
    dbname='data'
    user='postgres'
    host='localhost'
    password='postgres'
""")

In [157]:
cur = db.cursor()

In [162]:
cur.execute("select * from bundestag_votings_meta")
db.commit()

In [168]:
cur.fetchall()
db.commit()

ProgrammingError: no results to fetch

In [169]:
cur = db.cursor()
cur.execute("insert into bundestag_votings_individual VALUES('x',1,'y')")
db.commit()

In [170]:
cur = db.cursor()
cur.execute("""
    insert into bundestag_votings_meta
        (date, title, subtitle)
        VALUES (%s,%s,%s)
        returning id
""", (date,title,sub))
db.commit()

In [171]:
x = cur.fetchall()

In [166]:
db.commit()

In [202]:
"{a}+{b}".format(**{"a":1,"b":2})

'1+2'

In [206]:
tuple((t["name"],t["party"],t["type"]) for t in votings_data[0:2])

(('Aigner, Ilse', 'CDU/CSU', 1), ('Aumer, Peter', 'CDU/CSU', 1))

In [208]:
db_config = {"db":"data", "user":"postgres", "pw":"postgres", "host":"localhost"}

In [210]:
print("""
            dbname='{db}'
            user='{user}'
            host='{host}'
            password='{pw}'
        """.format(**db_config))


            dbname='data'
            user='postgres'
            host='localhost'
            password='postgres'
        


#Fetch Member-IDs for Names

In [20]:
import psycopg2
db = psycopg2.connect("""
    dbname='data'
    user='postgres'
    host='localhost'
    password='postgres'
""")

In [24]:
vd = votings_data[1:10]

In [25]:
vd

[{'name': 'Aumer, Peter', 'party': 'CDU/CSU', 'type': 1},
 {'name': 'Bär, Dorothee', 'party': 'CDU/CSU', 'type': 1},
 {'name': 'Bareiß, Thomas', 'party': 'CDU/CSU', 'type': 1},
 {'name': 'Barthle, Norbert', 'party': 'CDU/CSU', 'type': 1},
 {'name': 'Baumann, Günter', 'party': 'CDU/CSU', 'type': 1},
 {'name': 'Beck, Ernst-Reinhard', 'party': 'CDU/CSU', 'type': 1},
 {'name': 'Behrens, Manfred', 'party': 'CDU/CSU', 'type': 1},
 {'name': 'Bellmann, Veronika', 'party': 'CDU/CSU', 'type': 1},
 {'name': 'Bergner, Dr. Christoph', 'party': 'CDU/CSU', 'type': 1}]

In [32]:
vdx = ",".join(["('{}','{}',{})".format(v["name"],v["party"],v["type"]) for v in vd])

In [39]:
cur = db.cursor()
cur.execute("""
    create table new_table(
        name text, 
        party text, 
        type integer, 
        id_member integer)
    """)
db.commit()

cur.execute("insert into new_table (name, party, type) values " + vdx)
db.commit()

cur.execute("""
    update new_table as t
        set id_member = m.id
        from bundestag_members m
        where m.name = t.name
    """)
db.commit()

In [41]:
"{1},{0}".format(0,1)

'1,0'